In [1]:
import os
import sys
from pathlib import Path
import glob
PATH_TO_PARTGP = os.getenv("PATH_TO_PARTGP")
sys.path.append(os.path.abspath(PATH_TO_PARTGP))

import numpy
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import uproot

from weaver.utils.import_tools import import_module
from weaver.utils.dataset import SimpleIterDataset
from weaver.train import test_load
from weaver.utils.nn.tools import evaluate_classification

from utils.nn_utils.hook_handler import register_forward_hooks, remove_all_forward_hooks

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
training = False
load_model = True
datasets_path = os.getenv("PART_DATA")

In [3]:
path_to_network = (Path(PATH_TO_PARTGP) / 'models/networks/basic_mlp.py')

network_module = import_module(path_to_network, name='_network_module')

In [12]:
config_path = (Path(PATH_TO_PARTGP) / 'data_config/JetClass/MLP_JetClass.yaml')

data_config = SimpleIterDataset({}, str(config_path), for_training=training).config

model, model_info = network_module.get_model(data_config, num_layers = 3)

In [15]:
model

BasicMLP(
  (layers): Sequential(
    (0): Linear(in_features=7, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=64, bias=True)
    (7): ReLU()
    (8): Linear(in_features=64, out_features=2, bias=True)
  )
)

In [16]:
model_info

{'input_names': ['pf_points', 'pf_features', 'pf_vectors', 'pf_mask'],
 'input_shapes': {'pf_points': (1, 2, 128),
  'pf_features': (1, 7, 128),
  'pf_vectors': (1, 4, 128),
  'pf_mask': (1, 1, 128)},
 'output_names': ['softmax'],
 'dynamic_axes': {'pf_points': {0: 'N', 2: 'n_pf'},
  'pf_features': {0: 'N', 2: 'n_pf'},
  'pf_vectors': {0: 'N', 2: 'n_pf'},
  'pf_mask': {0: 'N', 2: 'n_pf'},
  'softmax': {0: 'N'}}}